In [4]:
import numpy as np
import os
from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skimage import io


In [7]:
os.chdir('histopathologic-cancer-detection/train')
os.getcwd()


'/home/arkadiusz/Dokumenty/Projekt NN/histopathologic-cancer-detection/train'

In [9]:
files_list = os.listdir()

In [11]:
test=[]
for file in files_list:
    test.append(io.imread(file, plugin='tifffile'))
test=np.array(test)

In [12]:
test.shape

(220025, 96, 96, 3)